Parte A

Levantar cluster en AWS con Hadoop y Pyspark
Nombre: cluster_ + la mátricula (número de estudiante) más chica

Acceso a JupyterHub
JupyterHub: https://master-public-dns-name:9443/
- u: `jovyan`
- p: `jupyter`

Librerías y sesión de Spark

In [1]:
# Crear sesión de SPARK y Cargar los datos 
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    year, to_date, countDistinct, 
    col, max as max_, row_number, 
    collect_list, array_contains, lag)
from pyspark.sql.window import Window
from pyspark.sql.types import StringType

BUCKET = "s3://proyecto-parcial-equipo-6/"

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1713968988527_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Carga el CSV en Spark

In [2]:
# Revisión de estructura de datos antes de tener los headers
# Inicia una sesión Spark.
spark = SparkSession.builder.appName("parteA").getOrCreate()

# Lee desde S3.
df = spark.read.csv(BUCKET + "clean/", header=False, inferSchema=True, sep='|')

# Muestra las primeras filas para asegurarte de que se ha cargado correctamente.
df.show(20)

# Muestra schema
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: date (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)

Carga el CSV en Spark, 
Adición de headers, 
Optimizar data types

In [3]:
# Renombrar las columnas del DataFrame.
df_with_headers = df.toDF("producto", "presentacion", "marca", "categoria", "catalogo", "precio", "fecha_registro", "cadena_comercial", "giro", "nombre_comercial", "direccion", "estado", "municipio", "latitud", "longitud")

# optimizar data type
df_with_headers = df_with_headers.withColumn("latitud", col("latitud").cast("float"))
df_with_headers = df_with_headers.withColumn("longitud", col("longitud").cast("float"))

# Muestra las primeras filas para asegurarte de que se han cargado correctamente con los nuevos encabezados.
df_with_headers.show(20)

# Muestra schema
df_with_headers.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- producto: string (nullable = true)
 |-- presentacion: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- catalogo: string (nullable = true)
 |-- precio: double (nullable = true)
 |-- fecha_registro: date (nullable = true)
 |-- cadena_comercial: string (nullable = true)
 |-- giro: string (nullable = true)
 |-- nombre_comercial: string (nullable = true)
 |-- direccion: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)

Guarda el CSV como parquet en S3, particionalo por catalogo. (Utiliza todos los trucos que consideres).

In [5]:
# Guarda el CSV como parquet en S3, particionalo por catalogo
df_with_headers.write.mode("overwrite").partitionBy("catalogo").parquet(BUCKET + "output/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Carga el parquet en Spark

In [6]:
# Carga del parquet en Spark
parquet_df = spark.read.parquet(BUCKET + "output/")

# Muestra las primeras filas para verificar.
parquet_df.show(20)

# Muestra schema
parquet_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- producto: string (nullable = true)
 |-- presentacion: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- precio: double (nullable = true)
 |-- fecha_registro: date (nullable = true)
 |-- cadena_comercial: string (nullable = true)
 |-- giro: string (nullable = true)
 |-- nombre_comercial: string (nullable = true)
 |-- direccion: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- latitud: float (nullable = true)
 |-- longitud: float (nullable = true)
 |-- catalogo: string (nullable = true)

¿Cuántos catálogos diferentes tenemos?

In [9]:
catalog_count_by_year = (parquet_df
    .groupBy(year("fecha_registro").alias("año"))  # Extrae el año y agrupa por este
    .agg(countDistinct("catalogo").alias("distinct_catalogs"))  # Cuenta los catálogos únicos
)

# Muestra el resultado.
catalog_count_by_year.show()

# Guarda la salida en el bucket de S3.
catalog_count_by_year.write.mode("overwrite").parquet(BUCKET + "preguntas/diff_catalogs/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------------+
| a?o|distinct_catalogs|
+----+-----------------+
|2018|               10|
|2019|               11|
|2020|               10|
|2021|               10|
|2022|               11|
|2023|               11|
|2024|               10|
+----+-----------------+

¿Cuáles son los 20 catálogos con más observaciones? Guarda la salida de este query en tu bucket de S3, lo necesitaremos más adelante.

In [11]:
# Encuentra los 20 catálogos con más observaciones.
top_catalogs = parquet_df.groupBy("catalogo").count().orderBy(col("count").desc()).limit(20)

# Muestra el resultado.
top_catalogs.show()

# Guarda la salida en el bucket de S3.
top_catalogs.write.mode("overwrite").parquet(BUCKET + "preguntas/top_catalogs/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------+
|           catalogo|   count|
+-------------------+--------+
|            basicos|46967421|
|       medicamentos|19223453|
|  electrodomesticos| 7175538|
| frutas y legumbres| 5041732|
|   utiles escolares| 2936010|
|           mercados| 2238684|
|           juguetes| 1432183|
|              pacic| 1079162|
|pescados y mariscos|  569632|
|           navideos|  236543|
|              tenis|   15768|
|        aeropuertos|     581|
+-------------------+--------+

¿Tenemos datos de todos los estados del país? De no ser así, ¿cuáles faltan?

In [12]:
# Lista todos los estados únicos presentes en el DataFrame.
unique_states = parquet_df.select("estado").distinct()

# Muestra los estados únicos.
unique_states.show()

# Opcional: Guarda la salida en el bucket de S3.
unique_states.write.mode("overwrite").csv(BUCKET + "preguntas/unique_states/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+
|         estado|
+---------------+
|ciudad de mxico|
|     tamaulipas|
|      zacatecas|
|       campeche|
|         yucatn|
|       veracruz|
|        morelos|
|     guanajuato|
|         sonora|
|       tlaxcala|
|        nayarit|
|        sinaloa|
|         oaxaca|
|       guerrero|
|   quintana roo|
|       quertaro|
| san luis potos|
|         puebla|
|        durango|
|estado de mxico|
+---------------+
only showing top 20 rows

In [13]:
# all_states es una lista de todos los estados posibles del país.
all_states = ['ciudad de mxico','tamaulipas','zacatecas','campeche','yucatn','veracruz','morelos',
              'guanajuato','sonora','tlaxcala','nayarit','sinaloa','oaxaca','guerrero',
              'quintana roo','quertaro','san luis potos','puebla','durango','estado de mxico',
              'jalisco','aguascalientes','coahuila de zaragoza','baja california sur','colima',
              'nuevo len','tabasco','chihuahua','baja california','michoacn de ocampo','chiapas',
              'hidalgo']

# Crea un DataFrame para la lista completa de estados.
all_states_df = spark.createDataFrame([(state,) for state in all_states], ["estado"])

# Realiza un anti join para encontrar los estados que faltan.
missing_states = all_states_df.join(unique_states, "estado", "left_anti")

# Muestra los estados que faltan.
missing_states.show()

# Opcional: Guarda la salida en el bucket de S3.
missing_states.write.mode("overwrite").csv(BUCKET + "preguntas/missing_states/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+
|estado|
+------+
+------+

¿Cuántas observaciones tenemos por estado?

In [14]:
# Agrupa por estado y cuenta las observaciones
observations_per_state = parquet_df.groupBy("estado").count().orderBy(col("count").desc())

# Muestra los resultados
observations_per_state.show()

# Opcional: Guarda la salida en el bucket de S3.
observations_per_state.write.mode("overwrite").csv(BUCKET + "preguntas/observations_per_state/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------+
|              estado|   count|
+--------------------+--------+
|     ciudad de mxico|17968798|
|     estado de mxico|12537548|
|             jalisco| 4182990|
|           nuevo len| 3358320|
|          guanajuato| 3330005|
|        quintana roo| 2821004|
|            veracruz| 2661838|
|            quertaro| 2534860|
|             tabasco| 2471506|
|           zacatecas| 2380067|
|              puebla| 2345346|
|              yucatn| 2331663|
|          tamaulipas| 1896217|
|              sonora| 1887619|
|            tlaxcala| 1810148|
|           chihuahua| 1798478|
|            campeche| 1765074|
|      san luis potos| 1718257|
|      aguascalientes| 1668545|
|coahuila de zaragoza| 1616519|
+--------------------+--------+
only showing top 20 rows

De cada estado obten: el número de catalogos diferentes por año, ¿ha aumentado el número de catálogos con el tiempo?

In [17]:
# Define una ventana particionada por estado y ordenada por año para análisis temporal.
windowSpec = Window.partitionBy("estado").orderBy("year")

# Preparar el DataFrame y calcular el número de catálogos distintos por estado y año, y el cambio año a año.
catalogs_per_state_year = (
    parquet_df
    # Asegurarse de que 'fecha_registro' está en formato de fecha correcto.
    .withColumn("fecha_registro", to_date("fecha_registro", "yyyy-MM-dd"))
    # Extraer el año y agrupar por este y por estado.
    .groupBy("estado", year("fecha_registro").alias("year"))
    .agg(countDistinct("catalogo").alias("num_catalogs"))
    # Calcular el número de catálogos del año anterior y el cambio respecto al año anterior.
    .withColumn("prev_year_catalogs", lag("num_catalogs", 1).over(windowSpec))
    .withColumn("change", col("num_catalogs") - col("prev_year_catalogs"))
    .select("estado", "year", "num_catalogs", "change")
    .orderBy("estado", "year")
)

# Muestra los resultados
catalogs_per_state_year.show()

# Opcional: Guarda la salida en el bucket de S3.
catalogs_per_state_year.write.mode("overwrite").csv(BUCKET + "preguntas/catalogs_per_state_year/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----+------------+------+
|             estado|year|num_catalogs|change|
+-------------------+----+------------+------+
|     aguascalientes|2018|           9|  NULL|
|     aguascalientes|2019|          11|     2|
|     aguascalientes|2020|           9|    -2|
|     aguascalientes|2021|          10|     1|
|     aguascalientes|2022|          11|     1|
|     aguascalientes|2023|          11|     0|
|     aguascalientes|2024|           9|    -2|
|    baja california|2018|          10|  NULL|
|    baja california|2019|          10|     0|
|    baja california|2020|           9|    -1|
|    baja california|2021|           9|     0|
|    baja california|2022|          11|     2|
|    baja california|2023|          11|     0|
|    baja california|2024|          10|    -1|
|baja california sur|2018|           9|  NULL|
|baja california sur|2019|          10|     1|
|baja california sur|2020|           9|    -1|
|baja california sur|2021|          10|     1|
|baja califor

Utilizando Spark contesta las siguientes preguntas a partir del catálogo que le tocó a tu equipo. Recuerda trabajar en el archivo con los datos particionados de otra manera tus queries van a tardar mucho.

¿Cuańtas marcas diferentes tiene tu categoría?

In [19]:
# Filtra por catalogo asignado
filtered_df = parquet_df.filter(col("catalogo") == "basicos")

# Selecciona la columna 'marca', encuentra valores únicos, y cuenta
unique_brands_count = filtered_df.select("marca").distinct().count()

print(f"Numero de marcas unicas en la catalogo: 'Basicos': {unique_brands_count}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Numero de marcas unicas en la catalogo: 'Basicos': 658

¿Cuál es la marca con mayor precio? ¿En qué estado?

In [20]:
# Encuentra la marca con el precio más alto y el estado correspondiente
max_price_info = filtered_df.orderBy(col("precio").desc()).select("marca", "precio", "estado").first()

print(f"La marca con el precio más alto es {max_price_info['marca']} con un precio de {max_price_info['precio']} en el estado de {max_price_info['estado']}.")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

La marca con el precio m?s alto es sin marca con un precio de 899.0 en el estado de tlaxcala.

¿Cuál es la marca con menor precio en CDMX? (en aquel entonces Distrito Federal)

In [23]:
# Filtra por estados que contengan la secuencia 'CIUDAD DE'
df_cdmx = filtered_df.filter(col("estado").contains("ciudad de"))

# Encuentra la marca con el precio más bajo y el estado correspondiente
min_price_info = df_cdmx.orderBy(col("precio").asc()).select("marca", "precio", "municipio").first()

print(f"La marca con el precio más bajo en Ciudad de México es {min_price_info['marca']} con un precio de {min_price_info['precio']}.")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

La marca con el precio m?s bajo en Ciudad de M?xico es s/m con un precio de 1.0.

¿Cuál es la marca con mayores observaciones?

In [24]:
# Agrupa por la columna 'marca' y cuenta el número de observaciones para cada marca
brand_counts = filtered_df.groupBy("marca").count()

# Ordena los resultados por el conteo en orden descendente y toma la primera fila
most_observed_brand = brand_counts.orderBy(col("count").desc()).first()

# Verifica si hay resultados antes de acceder al resultado
if most_observed_brand:
    print(f"La marca con el mayor número de observaciones es {most_observed_brand['marca']}, con {most_observed_brand['count']} observaciones.")
else:
    print("No se encontraron registros en el dataset.")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

La marca con el mayor n?mero de observaciones es s/m, con 2762155 observaciones.

¿Cuáles son el top 5 de marcas con mayor precio en cada estado? ¿Son diferentes?

In [25]:
# Definir una ventana ordenada por precio descendente dentro de cada estado
windowSpec = Window.partitionBy("estado").orderBy(col("precio").desc())

# Aplicar la ventana con un número de fila para identificar los 5 principales
state_brand_prices = filtered_df.withColumn("row_number", row_number().over(windowSpec)) \
    .filter(col("row_number") <= 5) \
    .select("estado", "marca", "precio", "row_number")

state_brand_prices.show()

# Para comparar las 5 listas principales en diferentes estados, las recopilamos e imprimimos agrupadas por estado.
top_brands_by_state = state_brand_prices.groupBy("estado") \
    .agg(collect_list("marca").alias("top_5_brands")) \
    .orderBy("estado")
top_brands_by_state.show(50, False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------+------+----------+
|             estado|     marca|precio|row_number|
+-------------------+----------+------+----------+
|     aguascalientes|  saba cup| 609.0|         1|
|     aguascalientes|  saba cup| 609.0|         2|
|     aguascalientes|  saba cup| 609.0|         3|
|     aguascalientes|  saba cup| 609.0|         4|
|     aguascalientes|  saba cup| 609.0|         5|
|    baja california|       s/m| 659.9|         1|
|    baja california|       s/m| 659.9|         2|
|    baja california|       s/m| 659.9|         3|
|    baja california|       s/m| 659.9|         4|
|    baja california|       s/m| 659.9|         5|
|baja california sur|diva cup 2| 690.0|         1|
|baja california sur|diva cup 2| 690.0|         2|
|baja california sur|diva cup 2| 690.0|         3|
|baja california sur|diva cup 2| 690.0|         4|
|baja california sur|diva cup 2| 690.0|         5|
|           campeche|  saba cup| 580.0|         1|
|           campeche|  saba cup

¿Cuáles son el top 5 de marcas con menor precio en CDMX? (en aquel entonces Distrito Federal)

In [26]:
# Define una ventana ordenada por precio ascendente dentro de cada estado que contiene "CIUDAD DE".
windowSpec = Window.partitionBy("estado").orderBy(col("precio").asc())

# Aplica la especificación de ventana con un número de fila para identificar el top 5 de precios más bajos.
lowest_price_brands = filtered_df.withColumn("row_number", row_number().over(windowSpec)) \
    .filter(col("row_number") <= 5) \
    .select("estado", "marca", "precio", "row_number")

# Muestra las 5 marcas con los precios más bajos en los estados que contienen "CIUDAD DE".
lowest_price_brands.show()

# Para comparar las listas del top 5 entre diferentes "CIUDAD DE", las recopilamos y las mostramos agrupadas por estado.
top_brands_by_state_lowest = lowest_price_brands.groupBy("estado") \
    .agg(collect_list("marca").alias("top_5_lowest_brands")) \
    .orderBy("estado")
top_brands_by_state_lowest.show(50, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----+------+----------+
|             estado|marca|precio|row_number|
+-------------------+-----+------+----------+
|     aguascalientes|  s/m|   1.0|         1|
|     aguascalientes|  s/m|   1.2|         2|
|     aguascalientes|  s/m|   1.2|         3|
|     aguascalientes|  s/m|   1.2|         4|
|     aguascalientes|  s/m|   1.2|         5|
|    baja california|  s/m|   1.5|         1|
|    baja california|  s/m|   1.5|         2|
|    baja california|  s/m|   1.5|         3|
|    baja california|  s/m|   1.5|         4|
|    baja california|  s/m|   1.5|         5|
|baja california sur|  s/m|  1.15|         1|
|baja california sur|  s/m|  1.15|         2|
|baja california sur|  s/m|  1.15|         3|
|baja california sur|  s/m|  1.15|         4|
|baja california sur|  s/m|   1.2|         5|
|           campeche|  s/m|   1.1|         1|
|           campeche|  s/m|   1.1|         2|
|           campeche|  s/m|   1.1|         3|
|           campeche|  s/m|   1.1|

¿Cuáles son el top 5 de marcas con mayores observaciones? ¿Se parecen a las de nivel por estado?

In [27]:
# Contar las ocurrencias de cada marca en todo el dataset
brand_occurrences = filtered_df.groupBy("marca").count()

# Ordenar las marcas por el número de observaciones en orden descendente y seleccionar el top 5
top_5_brands_observations = brand_occurrences.orderBy(col("count").desc()).limit(5)

# Mostrar el top 5 de marcas con mayores observaciones
top_5_brands_observations.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------+
|    marca|  count|
+---------+-------+
|      s/m|2762155|
|la costea|1311513|
|      fud|1151111|
|    bimbo| 819541|
|     lala| 750189|
+---------+-------+

In [28]:
# Buscar las marcas top 5 de observaciones en una lista
top_5_observation_list = [row['marca'] for row in top_5_brands_observations.collect()]

# Ahora creamos el DataFrame especificando el esquema
top_5_obs_df = spark.createDataFrame(top_5_observation_list, StringType()).withColumnRenamed("value", "marca")
top_5_obs_df.show()

# Realizar un join entre el DataFrame de top 5 por precio y el DataFrame de observaciones top 5
comparison_df = state_brand_prices.join(top_5_obs_df, "marca", "inner")

# Mostrar los resultados de la comparación
comparison_df.show()

# Comprobar cuántos estados tienen marcas que están en las top 5 de observaciones
distinct_states_with_top5_obs = comparison_df.select("estado").distinct().count()
print(f"Número de estados que tienen al menos una marca en común con las top 5 de observaciones: {distinct_states_with_top5_obs}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+
|    marca|
+---------+
|      s/m|
|la costea|
|      fud|
|    bimbo|
|     lala|
+---------+

+-----+--------------------+------+----------+
|marca|              estado|precio|row_number|
+-----+--------------------+------+----------+
|  s/m|     baja california| 659.9|         1|
|  s/m|     baja california| 659.9|         2|
|  s/m|     baja california| 659.9|         3|
|  s/m|     baja california| 659.9|         4|
|  s/m|     baja california| 659.9|         5|
|  s/m|coahuila de zaragoza| 659.9|         1|
|  s/m|coahuila de zaragoza| 659.9|         2|
|  s/m|coahuila de zaragoza| 659.9|         3|
|  s/m|coahuila de zaragoza| 659.9|         4|
|  s/m|coahuila de zaragoza| 659.0|         5|
|  s/m|              colima| 470.0|         1|
|  s/m|              colima| 470.0|         2|
|  s/m|              colima| 445.0|         3|
|  s/m|              colima| 399.0|         4|
|  s/m|             hidalgo| 539.0|         5|
|  s/m|             nayarit| 659.9|         1

¿Ha dejado de existir alguna marca durante los años que tienes? ¿Cuál? ¿Cuándo desapareció?

In [30]:
# Extraer el año de la columna 'registro' que tiene formato DD-MM-YYYY
# Convertir la columna de registro a tipo fecha y extraer el año
filtered_df = filtered_df.withColumn("fecha", to_date(col("fecha_registro"), "dd-MM-yyyy"))
filtered_df = filtered_df.withColumn("fecha_y", year(col("fecha")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
# Ventana para particionar por marca
window_spec = Window.partitionBy("marca")

# Encontrar el último año de aparición para cada marca
last_appearance = filtered_df.withColumn("last_year", max_("fecha_y").over(window_spec))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+----------------+------------------+------+--------------+--------------------+--------------------+--------------------+--------------------+--------+-----------+---------+----------+--------+----------+-------+---------+
|producto|        presentacion|           marca|         categoria|precio|fecha_registro|    cadena_comercial|                giro|    nombre_comercial|           direccion|  estado|  municipio|  latitud|  longitud|catalogo|     fecha|fecha_y|last_year|
+--------+--------------------+----------------+------------------+------+--------------+--------------------+--------------------+--------------------+--------------------+--------+-----------+---------+----------+--------+----------+-------+---------+
| yoghurt|bote 900 gr. natu...|danone. vitalnea|derivados de leche|  49.9|    2023-04-10|            chedraui|supermercado / ti...|chedraui sucursal...|carret. apizaco-h...|tlaxcala|    apizaco|19.412167|-98.133514| basicos|2023-04-10|   

In [32]:
# Seleccione columnas distintas de marca y último año para evitar filas duplicadas
distinct_last_appearance = last_appearance.select("marca", "last_year").distinct()
distinct_last_appearance.show(5)

distinct_last_appearance.write.mode("overwrite").parquet(BUCKET + "preguntas/marcas_que_se_fueron/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+---------+
|           marca|last_year|
+----------------+---------+
|          always|     2024|
|   azteca de oro|     2024|
|borges. original|     2024|
|danone. vitalnea|     2024|
|    de la fuente|     2024|
+----------------+---------+
only showing top 5 rows

Genera una gráfica de serie de tiempo por estado para la marca con mayor precio -en todos los años-, donde el eje equis es el año y el eje ye es el precio máximo.

In [36]:
# Seleccionar top marca (por precio máximo) por estado 
top_brands_by_state_filtered = top_brands_by_state.filter(col("row_number") == 1).select("estado", "marca")

# Obtener máximo precio por año de cada top marca (por precio máximo) de cada estado.
yseries_top_brands_by_state = (
    filtered_df
    .select("estado", "marca", "fecha_y", "precio")
    .groupBy("estado", "marca", "fecha_y")
    .agg(max("precio").alias("max_precio"))
    .join(top_brands_by_state_filtered, ["estado", "marca"], "inner")
)

# Mostrar resultado
yseries_top_brands_by_state.show()

# Guarda la salida en el bucket de S3.
#yseries_top_brands_by_state.write.mode("overwrite").parquet(BUCKET + "preguntas/yserie_estado_max_price/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…